In [45]:
# coding: utf-8
import traceback
import pygeoip
import threading
import socket
import sys
import hashlib
import datetime
import time
import json

import metautils
from bencode import bencode, bdecode
geoip = pygeoip.GeoIP('GeoIP.dat')

def decode(encoding, s):
    if type(s) is list:
        s = ';'.join(s)
    u = s
    for x in (encoding, 'utf8', 'gbk', 'big5'):
        try:
            u = s.decode(x)
            return u
        except:
            pass
    return s.decode(encoding, 'ignore')

def decode_utf8(encoding, d, i):
    if i+'.utf-8' in d:
        return d[i+'.utf-8'].decode('utf8')
    return decode(encoding, d[i])

def parse_metadata(data):
    info = {}
    encoding = 'utf8'
    try:
        torrent = bdecode(data)
        if not torrent.get('info').get('name'):
            print 'not get name'
            return None
    except:
        return None
    try:
        info['create_time'] = datetime.datetime.fromtimestamp(float(torrent['creation date']))
    except:
        info['create_time'] = datetime.datetime.utcnow()
    if torrent.get('encoding'):
        encoding = torrent['encoding']
    if torrent.get('announce'):
        info['announce'] = decode_utf8(encoding, torrent, 'announce')
    if torrent.get('comment'):
        info['comment'] = decode_utf8(encoding, torrent, 'comment')[:200]
    if torrent.get('publisher-url'):
        info['publisher-url'] = decode_utf8(encoding, torrent, 'publisher-url')
    if torrent.get('publisher'):
        info['publisher'] = decode_utf8(encoding, torrent, 'publisher')
    if torrent.get('created by'):
        info['creator'] = decode_utf8(encoding, torrent, 'created by')[:15]

    if 'info' in torrent:
        detail = torrent['info'] 
    else:
        detail = torrent
    info['name'] = decode_utf8(encoding, detail, 'name')
    if 'files' in detail:
        info['files'] = []
        for x in detail['files']:
            if 'path.utf-8' in x:
                v = {'path': decode(encoding, '/'.join(x['path.utf-8'])), 'length': x['length']}
            else:
                v = {'path': decode(encoding, '/'.join(x['path'])), 'length': x['length']}
            if 'filehash' in x:
                v['filehash'] = x['filehash'].encode('hex')
            info['files'].append(v)
        info['length'] = sum([x['length'] for x in info['files']])
    else:
        info['length'] = detail['length']
    info['data_hash'] = hashlib.md5(detail['pieces']).hexdigest()
    if 'profiles' in detail:
        info['profiles'] = detail['profiles']
    return info



In [50]:
#coding: utf8
import re
import codecs

if __name__ == "__main__":
#需要读取的文件名称放到这里
    fpath = "/home/work/data/dht/torrent-test/A1DEC5568905B8A1C67B0E4C5BA4A26B8B1FD50C.torrent"
    data = open(fpath).read()
    info = parse_metadata(data)
    print info
    print 'name : ', info.get('name')

{'files': [{'path': u'RARBG.com.txt', 'length': 33}, {'path': u'poster.jpg', 'length': 47741}, {'path': u'screens.jpg', 'length': 651105}, {'path': u'these.16.01.11.addison.ryder.and.alix.lynx.mp4', 'length': 1301897030}], 'length': 1302595909, 'create_time': datetime.datetime(2016, 2, 14, 14, 4, 29, 187554), 'name': u'TheStripperExperience.16.01.11.Addison.Ryder.And.Alix.Lynx.XXX.1080p.MP4.KTR', 'data_hash': '7a77f9f908437dd9f78831d7aa72923a'}
name :  TheStripperExperience.16.01.11.Addison.Ryder.And.Alix.Lynx.XXX.1080p.MP4.KTR


In [19]:
print '\x83\xb7Zy\xac\xa3\xfc\x9e\xb7\xb6\x16\x87\xa1Q\x1b\xf0c\x11[\xba\xb1\xeb\xb7k\xa04D\xff\r\x15\x14P7M\xf8\xb1cq\x92\xab\n\xdc\x98\x16\xa0\xe0\xd7V\xfd\x94\xdc\xaa\xf8\x80\xb9'

P7M��cq��
ܘ���V��ܪ���


In [22]:
u = u'脚本'
print repr(u)

u'\u811a\u672c'


In [23]:
v = '脚本'
print repr(v)

'\xe8\x84\x9a\xe6\x9c\xac'


In [26]:
s = u.encode('utf-8')
print repr(s)

'\xe8\x84\x9a\xe6\x9c\xac'


In [27]:
t = v.decode('utf-8')
print repr(t)

u'\u811a\u672c'


In [61]:
# coding: utf-8
folder_name = '/home/work/data/dht/torrent-beijing'
import os
output_path = '/home/work/data/dht/beijing.txt'

output = codecs.open(output_path, 'w', encoding='utf-8')
for _torrent_file in os.listdir(folder_name):
    data = open(folder_name +'/'+ _torrent_file).read()
    info = parse_metadata(data)
    _line = _torrent_file + " : " + info.get('name')
    output.write(_line+"\n")
print 'over'

over


In [74]:
hash_path = '/home/work/data/dht/hash-beijing/HASH.log-20160113'
lines = open(hash_path)
freq_dict = {}
for _line in lines:
    #print _line
    _array = _line.split()
    _seed = _array[7]
    _ip = _array[9]
    #print _seed, " at ", _ip
    if _seed in freq_dict:
        freq = freq_dict[_seed]
        freq_dict[_seed] = freq+1
    else:
        freq_dict[_seed]= 1

for _seed in freq_dict:
    frq = freq_dict[_seed]
    if frq > 1:
        print _seed, " with ", frq


DF0DFAD080A3B60DA7410E7222A675220DF34B48  with  2
EE3560476D774445BF3EADFD7DEF18A0956439D6  with  2
12ABB83491320A619D7C99AD82BDF6CE150CB5AB  with  2
A4800C3B0E2BE21CFEB16F3A61D5F8CA004A1241  with  2
328210AEC9D8395867688B9F1CEB587500001881  with  2
900B30D6AE529049F1F1BBE9EBB3A6DB3C870CE1  with  2
E52622F1F1BBE9EBB3A6DB3C870C3E99245E0D90  with  2
6176FD9A192B10578EE878293F0A603CB186EC34  with  2
933ABE17A7DF738BD7E89C9B40772008094302CF  with  2
A0AE305877B12E6916A8149B181A9AB8F0202506  with  2
591CE2C0ED163ED12667913FE650A4DFBCEDAF55  with  2
C43B7E4392C83EAD73D4CBC2B7D17C6665E6D023  with  2
FBFADF79F175D8990DBE9E70285AC6FB93C7A7ED  with  3
3E65A6FC79C603920095A3C572640F3735103640  with  2
87483CECCE3AAB556E41B5F4813985F3300686F6  with  2
D098E9AE529049F1F1BBE9EBB3A6DB3C870C3E6C  with  2
58DEFC7C4A54B36B51CF22A36A0FD3F75A9019D7  with  2
0D4AE9AA452BD5120D7A476ACD2E3A3407BBD0F0  with  20
479837E4EBB88D19F2711E5F8996F4BACF9A4400  with  3
6321052892E33E372CD896A07E6C66439F8299F8  with  2

In [81]:
dic = {'a':1, 'b':2, 'c':1, 'd':4}
print dic

{'a': 1, 'c': 1, 'b': 2, 'd': 4}


In [82]:

dic_array = sorted(dic.iteritems(), key=lambda d:d[1], reverse=True)
print dic_array

[('d', 4), ('b', 2), ('a', 1), ('c', 1)]
